# Brute force
Just check all permutations\
(+) can be found global optimum\
(-) need to check n! permutations -> too long; many resources needed

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
import random
import time

import itertools, math

In [2]:
df = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')

In [3]:
df

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


In [6]:
words = df.loc[0,"text"].split()
all_permutations = list( itertools.permutations(words) )[::-1]
len(all_permutations)

3628800

In [7]:
from metric import PerplexityCalculator
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Stop algorithm after 2 hours of work (because it can work too long time)

In [10]:
PERM_CT = math.factorial(10)
BATCH_SIZE = 64
TIME_LIMIT = 2 * 60 * 60  # 2 hours in seconds

best_score = 1e6
best_text = ""

perms = []

start_time = time.time()  # Record the start time

for i, p in enumerate(all_permutations):
    perms.append(" ".join(p))
    if (len(perms) == BATCH_SIZE) or (i == PERM_CT - 1): 
        p = scorer.get_perplexity(perms)
        if np.min(p) < best_score:
            best_score = np.min(p)
            best_text = perms[np.argmin(p)]
            print(f"New best = {best_score} with '{best_text}'")
        perms = []
    if i % 10_000 == 0: 
        print(f"Completed computing {i} perplexities.")
    if best_score < 475: 
        print("Stopping early because we found optimal!")
        break
    
    # if elapsed time exceeds 2 hours -> break
    elapsed_time = time.time() - start_time
    if elapsed_time > TIME_LIMIT:
        print("Stopping because the time limit of 2 hours was exceeded.")
        break

print("Done.")

Completed computing 0 perplexities.
New best = 2258.9519595596075 with 'scrooge reindeer ornament mistletoe gingerbread family advent fireplace chimney elf'
New best = 2064.849249361922 with 'scrooge reindeer ornament mistletoe elf gingerbread family advent fireplace chimney'
New best = 1978.0044264076564 with 'scrooge reindeer ornament gingerbread family advent fireplace chimney elf mistletoe'
New best = 1843.7028557527542 with 'scrooge reindeer mistletoe ornament gingerbread family advent fireplace chimney elf'
New best = 1759.2735550257069 with 'scrooge reindeer mistletoe gingerbread ornament family advent fireplace chimney elf'
New best = 1665.646724298566 with 'scrooge reindeer mistletoe gingerbread fireplace chimney elf advent family ornament'
New best = 1583.1748363626543 with 'scrooge reindeer mistletoe gingerbread family elf chimney fireplace advent ornament'
New best = 1464.1973685237601 with 'scrooge reindeer mistletoe gingerbread family advent fireplace chimney elf ornament

In [13]:
df.loc[0,"text"] = best_text
best_text, best_score

('scrooge reindeer mistletoe elf gingerbread chimney fireplace advent family ornament',
 1375.486134528337)